# Projekt: symulacja automatu komórkowego reprezentującego pożar lasu (P6)
## Adam Trentowski - DSwP 1 - 162602

> <b>Cel</b>: Automat ma rozpoczynać pracę na losowym rozmieszczeniu elementów reprezentujących stany. 
Następnie w losowo wybranym miejscu należy zainicjować pożar. Ustalić, że symulacja kończy się, gdy spłonie ostatnie z płonących drzew.
Wykorzystać następujące stany (można zdefiniować dodatkowe):
• drzewo,
• płonace drzewo,
• spalone drzewo,
• woda.
> 
> Zasady ewolucji, np.:
• drzewo staje się płonącym drzewem z prawdopodobieństwem p, jeśli ma w sąsiedztwie płonace drzewo,
• płonące drzewo w następnej generacji staje się spalonym drzewem,
• spalone drzewo odnawia się po k iteracjach,
• samozapłon drzewa następuje z pewnym prawdopodobieństwem (odpowiednio małe),
• uwzględnić wodę, która stanowi barierę dla ognia,
• uwzględnić wiatr zmieniający prawdopodobieństwa rozprzestrzeniania się pożaru w różnych kierunkach; kierunek powinien zmieniać się co kilka iteracji,
• inne – według uznania.
> 
> Uwaga: określone drzewo (ustalona komórka) zapala się w zależności od stanu drzew (komórek) sąsiednich, a nie płonce drzewo zapala drzewa sąsiednie.

## Krótki opis
Poniższa symulacja pożaru lasu to model pokazujący proces rozprzestrzeniania się ognia w lesie w zależności od różnych czynników (np. wiatru, samozapłonu drzewa, zalesienia). Głównym celem jest wizualizacja, jak ogień może się rozprzestrzeniać w lesie, a także pokazanie jak wpływają na to zmienne środowiskowe.

## Metoda
Symulacja bazuje na automacie komórkowym, czyli modelu składającym się z siatki komórek, w której każda komórka przyjmuje określony stan. Zmiana stanu komórki w czasie jest uzależniona od jej stanu i stanu jej sąsiadów według określonych powyżej reguł. Poniższy projekt jest automatem komórkowym dwuwymiarowym (z sąsiedztwem Moore'a) o strukturze zamkniętej (krawędzie są traktowane jak woda).
* Sąsiedztwo Moore'a – dla danej komórki bierze się pod uwagę jej 8 sąsiadów w układzie 3x3 (więc w tym również przekątne).

## Import bibliotek

* <b>Python: 3.12.3</b>
* <b>NumPy: 2.1.3</b> (instalacja: pip install numpy==2.1.3)
* <b>Matplotlib: 3.9.2</b> (instalacja: pip install matplotlib==3.9.2)

In [ ]:
%matplotlib notebook
from enum import Enum
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Struktura modelu
Model lasu składa się z siatki, na której mogą występować następujące "obiekty leśne":
</br>
* <b>WATER</b> (Woda):
    Komórka reprezentująca obszar wodny. Nie ulega żadnym zmianom.
    Kolor: Niebieski.
</br>

* <b>TREE</b> (Drzewo):
    Rosnące, niezapalone drzewo. Może ulec zapłonowi przez samozapłon lub przez sąsiadujące płonące drzewa z pewnym prawdopodobieństem.
    Kolor: Zielony.
</br>

* <b>BURNING_TREE</b> (Płonące drzewo):
    Drzewo, które w danej iteracji płonie. W kolejnej iteracji staje się drzewem spalonym.
    Kolor: Czerwony.
</br>

* <b>BURNT_TREE</b> (Spalone drzewo):
    Drzewo, które zostały całkowicie spalone. Po określonym czasie może się odrodzić z pewnym prawdopodobieństem.
    Kolor: Czarny.
</br>

* <b>REBORN_TREE</b> (Odrodzone drzewo):
    Drzewo, które odrodziło się po spaleniu. Jest tym samym co zwykłe drzewo (TREE), różni się jedynie kolorem na wizualizacji.
    Kolor: Jasno-zielony.

In [ ]:
class ForestObject(Enum):
    WATER = 0
    TREE = 1
    BURNING_TREE = 2
    BURNT_TREE = 3
    REBORN_TREE = 4

In [ ]:
COLOR_MAP = {
            ForestObject.WATER.value: [0.07, 0.4, 0.69],          # Niebieski
            ForestObject.TREE.value: [0.0, 0.5, 0.0],             # Zielony
            ForestObject.BURNING_TREE.value: [0.73, 0.05, 0.04],  # Czerwony
            ForestObject.BURNT_TREE.value: [0.04, 0.04, 0.04],    # Czarny
            ForestObject.REBORN_TREE.value: [0.0, 0.8, 0.0],      # Jasno-zielony
        }

## Wiatr
Wiatr wpływa na prawdopodobieństwo zapłonu od sąsiadujących drzew. </br>
W modelu występuje wiatr w 4 kierunkach:
* Północnym (N ↑)
* Wschodnim (E →)
* Południowym (S ↓)
* Zachodnim (W ←)
</br>

Każdy kierunek wiatru ma taki sam wpływ na modyfikację prawdopodobieństwa zapłonu drzewa, różni się jedynie obszar, na który ten wpływ jest nakierowany.

In [ ]:
class Wind(Enum):
    N = [[0.25, 0.30, 0.25], 
         [0.50, 1.00, 0.50], 
         [1.25, 2.00, 1.25]]
    
    E = [[1.25, 0.50, 0.25], 
         [2.00, 1.00, 0.30], 
         [1.25, 0.50, 0.25]]
    
    S = [[1.25, 2.00, 1.25], 
         [0.50, 1.00, 0.50], 
         [0.25, 0.30, 0.25]]
    
    W = [[0.25, 0.50, 1.25], 
         [0.30, 1.00, 2.00], 
         [0.25, 0.50, 1.25]]

In [ ]:
WIND_ARROWS = {
        'N': '↑',
        'E': '→',
        'S': '↓',
        'W': '←'
    }

## Implementacja automatu komórkowego
Implementacja modelu opiera się na klasie <b>Forest</b>. Podczas tworzenia obiektu tej klasy można zdefiniować następujące parametry środowiskowe:
</br>
* <b>size</b> (rozmiar lasu) - określa rozmiar siatki (N x M)
* <b>afforestation</b> (zalesienie) - określa procent powierzchni siatki, jaka ma być zajmowana przez drzewa
* <b>p_ignition</b> (prawdopodobieństwo zapłonu) - określa prawdopodobieństwo zapłonu drzewa od sąsiadującego płonącego drzewa
* <b>tree_regeneration_time</b> (czas odrodzenia drzewa) - określa ile co najmniej musi upłynąć iteracji zanim spalone drzewo będzie mogło się odrodzić
* <b>p_tree_regeneration</b> (prawdopodobieństwo odrodzenia drzewa) - określa prawdopodobieństwo odrodzenia spalonego drzewa (po upłynięciu tree_regeneration_time iteracji)
* <b>p_self_ignition</b> (prawdopodobieństwo samozapłonu) - określa prawdopodobieństwo samozapłonu drzewa
* <b>wind_direction</b> (początkowy kierunek wiatru) - określa kierunek wiatru na początku symulacji 
* <b>wind_change_interval</b> (odstęp czasu między zmianą kierunku wiatru) - określa liczbę iteracji, jaka upływa między zmianami kierunku wiatru
</br>

Jeśli któryś z powyższych parametrów nie zostanie określony, zostanie zastosowana wartość domyślna (opisana dokładnie w dokumentacji metody \_\_init\_\_), która została dobrana intuicyjnie na bazie prób i błędów.
</br>

Poza tym, klasa <b>Forest</b> zawiera również licznik iteracji <i>self.iterations = -10</i> inicjowany z wartością -10. Spowodowane jest to problemem z powiększającym się oknem animacji. Przeskok uniemożliwiał zaobserowowanie kilku pierwszych stanów automatu (dlatego, pierwsze 10 iteracji wykonywane jest "na pusto").
</br>

Klasa <b>Forest</b> zawiera następujące metody:
</br>
- <b>_ignite()</b> - metoda zmieniająca losowe drzewo w płonące drzewo
- <b>update()</b> - metoda odpowiedzialna za generowanie kolejnych stanów automatu. Działanie opiera się na tworzeniu nowej siatki, która powstaje na podstawie aktualnego stanu każdej komórki oraz jej sąsiadów. W trakcie aktualizacji uwzględniane są reguły:
  - Reguła zapłonu (zależna od sąsiadów): Drzewo może zapłonąć, jeśli w sąsiedztwie ma płonące drzewo. Prawdopodobieństwo zapłonu jest modyfikowane przez kierunek wiatru.
    - Modyfikator - wiatr: Wiatr wpływa na rozprzestrzenianie się ognia, zwiększając lub zmniejszając prawdopodobieństwo zapłonu drzewa w zależności od kierunku.
  - Reguła samozapłonu: Istnieje niewielkie prawdopodobieństwo, że drzewo zapali się samo.
  - Reguła spalania: Płonące drzewo zmienia się w spalone drzewo w kolejnej iteracji.
  - Reguła odrodzenia: Spalone drzewo może się odrodzić po upływie określonego czasu z określonym prawdopodobieństwem.
  - Reguła zmiany wiatru: Kierunek wiatru zmienia się losowo co określoną liczbę iteracji.
  Wynikowa siatka zastępuje poprzedni stan automatu, a proces powtarza się w kolejnych iteracjach. Ponadto, w iteracji nr. 0 jeśli istnieje drzewo, następi pewny zapłon.
- <b>animate(frames)</b> -  metoda tworząca animację kolejnych stanów automatu komórkowego, z parametrem frames, który:
  - jeśli nie jest ustawiony: animacja kończy się, gdy wszystkie drzewa przestaną płonąć
  - jeśli jest ustawiony: animacja trwa podaną liczbę iteracji
  Działanie:
  - tworzona jest siatka RGB odpowiadająca stanom komórek
  - funkcja wewnętrzna render_frame() aktualizuje kolory siatki i wyświetla tytuł z informacją o bieżącej iteracji i kierunku wiatru
  - funkcja wewnętrzna update_animation(frame) generuje nowy stan automatu i sprawdza warunki zakończenia animacji
  Animacja jest realizowana przy pomocy FuncAnimation z biblioteki Matplotlib i wyświetlana w oknie graficznym.

In [ ]:
class Forest:
    def __init__(self, 
                 size: tuple=(30, 30), 
                 afforestation: float=80.0, 
                 p_ignition: float=75.0, 
                 tree_regeneration_time: int=15, 
                 p_tree_regeneration: float=80.0, 
                 p_self_ignition: float=0.005,
                 wind_direction: Wind=random.choice(list(Wind)),
                 wind_change_interval: int=8):
        """
        Tworzy nowy las z podanymi parametrami.
        
        :param size: rozmiar lasu (domyślnie (15, 15))
        :param afforestation: zalesienie (domyślnie 70%)
        :param p_ignition: prawdopodobieństwo zapłonu od sąsiadującego drzewa (domyślnie 75%)
        :param tree_regeneration_time: liczba iteracji, po których spalone drzewo się odrodzi (domyślnie 15)
        :param p_tree_regeneration: prawdopodobieństwo odrodzenia się drzewa po upłynięciu tree_regeneration_time (domyślnie 80 %)
        :param p_self_ignition: prawdopodobieństwo samozapłonu drzewa (domyślnie 0,005%)
        :param wind_direction: (początkowy) kierunek wiatru (domyślnie losowy)
        :param wind_change_interval: liczba iteracji, po których wiatr zmieni kierunek (domyślnie 8)
        """
        afforestation_p = afforestation / 100
        self.p_ignition = p_ignition / 100
        self.p_self_ignition = p_self_ignition / 100
        self.tree_regeneration_time = tree_regeneration_time
        self.p_tree_regeneration = p_tree_regeneration / 100
        self.grid = np.random.choice([ForestObject.WATER.value, ForestObject.TREE.value], size=size, p=[1 - afforestation_p, afforestation_p])
        self.padded_calendar_grid = np.pad(np.full(size, -1), pad_width=1, mode='constant', constant_values=-1)
        self.wind_direction = wind_direction
        self.wind_change_interval = wind_change_interval
        self.iterations = -10  # początkowy przeskok okiena z animacją nie pozwala na zaobserwowanie początkowego przebiegu symulacji
        
        
    def _ignite(self):
        """
        Wybiera losowy punkt, który jest drzewem i zmienia go w płonące drzewo.
        """
        tree_indexes = np.argwhere(self.grid == ForestObject.TREE.value)
        
        if len(tree_indexes) > 0:
            x, y = random.choice(tree_indexes)
            self.grid[x, y] = ForestObject.BURNING_TREE.value
    
    
    def update(self):
        """
        Zasymulowanie kolejnej jednostki czasu (przejście do kolejnego stanu).
        W pierwszej iteracji nastąpi zapłon.
        
        - Płonące drzewa (BURNING_TREE) zmieniają się w spalone drzewa (BURNT_TREE).
        - Drzewa (TREE) mogą zapłonąć z prawd. p_ignition jeśli sąsiadują z płonącym drzewem (BURNING_TREE).
        """ 
        padded_grid = np.pad(self.grid, pad_width=1, mode='constant', constant_values=ForestObject.WATER.value)
        new_grid = padded_grid.copy()
        wind_modifiers = self.wind_direction.value
        
        for row in range(1, padded_grid.shape[0] - 1):
            for col in range(1, padded_grid.shape[1] - 1):
                if self.iterations <= 0:
                    continue
                
                # zapłon od sąsiadującego drzewa (modyfikowany przez wiatr) / samozapłon
                if padded_grid[row, col] ==  ForestObject.TREE.value or padded_grid[row, col] == ForestObject.REBORN_TREE.value:
                    neighbors = padded_grid[row-1:row+2, col-1:col+2]
                    is_on_fire = False
                    for i in range(3):
                        for j in range(3):
                            if neighbors[i, j] == ForestObject.BURNING_TREE.value:
                                modified_p_ignition = self.p_ignition * wind_modifiers[i][j]
                                if np.random.rand() < modified_p_ignition:
                                    new_grid[row, col] = ForestObject.BURNING_TREE.value
                                    is_on_fire = True
                    if not is_on_fire:
                        if np.random.rand() < self.p_self_ignition:
                            new_grid[row, col] = ForestObject.BURNING_TREE.value
                
                # spalenie drzewa
                if padded_grid[row, col] == ForestObject.BURNING_TREE.value:
                    new_grid[row, col] = ForestObject.BURNT_TREE.value
                    self.padded_calendar_grid[row, col] = self.iterations
                
                # odrodzenie drzewa
                if padded_grid[row, col] == ForestObject.BURNT_TREE.value:
                    if self.padded_calendar_grid[row, col] != -1 and self.iterations - self.padded_calendar_grid[row, col] >= self.tree_regeneration_time:
                        if np.random.rand() < self.p_tree_regeneration:
                            new_grid[row, col] = ForestObject.REBORN_TREE.value
                        
        self.grid = new_grid[1:-1, 1:-1]
        
        if self.iterations == 0:
            self._ignite()
            
        if self.iterations > 0 and self.iterations % self.wind_change_interval == 0:
            self.wind_direction = random.choice(list(Wind))
        
        self.iterations += 1
   
   
    def animate(self, frames=None):
        """
        Tworzy animację kolejnych stanów lasu.
    
        :param frames:
        - jeśli nie jest ustawione: animacja zakończy się, gdy ostatnie drzewo przestanie płonąć.
        - jeśli jest ustawione: animacja będzie trwać podaną liczbę iteracji.
        """
        rgb_grid = np.zeros((*self.grid.shape, 3))
    
        fig, ax = plt.subplots(figsize=(7, 7))
        img = ax.imshow(rgb_grid)
        ax.axis('off')
        
        def render_frame():
            for value, color in COLOR_MAP.items():
                rgb_grid[self.grid == value] = color
                
            img.set_data(rgb_grid)
            i = self.iterations
            i = 0 if i < 0 else i
            ax.set_title(f"Iteracja: {i}, Wiatr: {self.wind_direction.name} {WIND_ARROWS[self.wind_direction.name]}")
        
        def update_animation(frame):
            if frames is None and self.iterations > 0 and ForestObject.BURNING_TREE.value not in self.grid:
                ani.event_source.stop()
                return img,
            
            if frames is not None and self.iterations >= frames:
                ani.event_source.stop()
                return img,
            
            self.update()
            render_frame()
            return img,
    
        render_frame()
    
        ani = FuncAnimation(fig, update_animation, frames=frames, interval=50, blit=False, cache_frame_data=False)
        plt.show()

## Wizualizacja symulacji

In [ ]:
size = (50, 50)
afforestation = 95.0
p_ignition = 50.0
tree_regeneration_time = 25
p_tree_regeneration = 48.0
p_self_ignition = 0.003
# wind_direction = Wind.N
wind_change_interval = 30

forest = Forest(
    size=size, 
    afforestation=afforestation, 
    p_ignition=p_ignition, 
    tree_regeneration_time=tree_regeneration_time, 
    p_tree_regeneration=p_tree_regeneration, 
    p_self_ignition=p_self_ignition, 
    # wind_direction=wind_direction, 
    wind_change_interval=wind_change_interval
)

forest.animate()

## Wnioski
W modelu pożaru lasu występuje kilka źródeł losowości:
- Losowy zapłon drzew - nie można przewidzieć, które drzewa zapłoną przez płonących sąsiadów, można natomiast stwierdzić, które <b>nie</b> zapłoną przez płonących sąsiadów.
- Losowy samozapłon drzew – nie można przewidzieć, które drzewa zapłoną, uniemożliwia możliwość stwierdzenia, które drzewa nie zapłoną.
- Losowy wybór kierunku wiatru – wpływa na rozprzestrzenianie się ognia.
- Odradzanie się drzew – proces regeneracji nie jest deterministyczny.

Z tego powodu nie można przewidzieć przyszłości automatu komórkowego ani określić dokładnego wzorca spalania. Jednocześnie, z powodu losowości, nie można jednoznacznie wskazać źródła pożaru na podstawie finalnego stanu automatu.